In [1]:
# dataset test
from datasets import load_dataset

dataset= load_dataset("albertvillanova/legal_contracts", trust_remote_code=True)

legal_contracts.py:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

c:\Users\82106\anaconda3\envs\iclr\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\82106\.cache\huggingface\hub\datasets--albertvillanova--legal_contracts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


contracts.tar.gz:   0%|          | 0.00/9.25G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/68 [00:00<?, ?it/s]

In [1]:
from datasets import load_dataset

contracts = load_dataset(
    "albertvillanova/legal_contracts",
    split="train",          # 기본 split
    streaming=True,         # 메모리 절약, 한 문서씩 읽기
    trust_remote_code=True  # 사용자 정의 로더 스크립트 실행 허용
)

In [2]:
print(next(iter(contracts))["text"][:500])   # 앞 500자만 보기


QuickLinks -- Click here to rapidly navigate through this document





AMENDED AND RESTATED
EMPLOYMENT AND NONCOMPETITION AGREEMENT

    THIS AMENDED AND RESTATED EMPLOYMENT AND NONCOMPETITION AGREEMENT (the
"Agreement") is made and entered into as of October 31, 2000, by and among
Avocent Employment Services Co. (formerly known as Polycon Investments, Inc.), a
Texas corporation ("Employer"), Avocent Corporation, a Delaware corporation, and
R. Byron Driver (the "Employee").


RECITALS

    WHER


In [3]:
contracts = load_dataset(
    "albertvillanova/legal_contracts",
    split="train",
    trust_remote_code=True      # 커스텀 스크립트
)                               # 약 9 GB, 꽤 오래 걸림


Loading dataset shards:   0%|          | 0/68 [00:00<?, ?it/s]

In [6]:
print(len(contracts))        # ≈ 106_000
contracts.features    # => {'text': Value('string')}


650833


{'text': Value(dtype='string', id=None)}

In [11]:
print(type(contracts))
# 출력이 <class 'datasets.iterable_dataset.IterableDataset'> 이라면 → streaming 모드!


<class 'datasets.arrow_dataset.Dataset'>


In [13]:
contracts = load_dataset(
    "albertvillanova/legal_contracts",
    split="train",
    streaming=False,        # ★ 중요: map/인덱싱 가능
    trust_remote_code=True
)
print(len(contracts), "rows")      # 정상 길이 출력 확인


650833 rows


In [14]:
import re, dateparser

TITLE_RE   = re.compile(r"^\s*(.+?)\n", re.S)
DATE_RE    = re.compile(r"\b\d{1,2}\s+\w+\s+\d{4}\b", re.S)
PARTIES_RE = re.compile(r"between\s+(.*?)\s+and\s+(.*?)[\.,\n]", re.I | re.S)

def parse_contract(ex):
    raw = ex["text"]
    try:
        title = TITLE_RE.search(raw).group(1).strip()
    except AttributeError:
        title = None

    m_date = DATE_RE.search(raw)
    date_iso = None
    if m_date:
        dt = dateparser.parse(m_date.group(0), settings={"DATE_ORDER": "DMY"})
        if dt:
            date_iso = dt.date().isoformat()

    m_par = PARTIES_RE.search(raw)
    p1, p2 = (m_par.group(1).strip(), m_par.group(2).strip()) if m_par else (None, None)

    return {"title": title, "date": date_iso, "party1": p1, "party2": p2}


In [15]:
sample_ok = parse_contract(contracts[0])   # 이제 인덱싱 가능
print(sample_ok)


{'title': 'QuickLinks -- Click here to rapidly navigate through this document', 'date': None, 'party1': None, 'party2': None}


In [16]:
parsed = contracts.map(
    parse_contract,
    remove_columns=["text"],   # 원문 필요 없으면 제거
    desc="Parsing contracts"
)
print(parsed[0])

parsed.save_to_disk("contracts_parsed")    # 전처리 결과 저장


Parsing contracts: 100%|██████████| 650833/650833 [15:28<00:00, 700.72 examples/s] 


{'title': 'QuickLinks -- Click here to rapidly navigate through this document', 'date': None, 'party1': None, 'party2': None}


Saving the dataset (1/1 shards): 100%|██████████| 650833/650833 [00:00<00:00, 2749333.72 examples/s]


In [19]:
from datasets import load_dataset

contracts = load_dataset(
    "albertvillanova/legal_contracts",
    split="train",
    trust_remote_code=True,
    streaming=False
)

# (1) 타입 & 행 수
print(type(contracts), len(contracts))
# → <class 'datasets.arrow_dataset.Dataset'> 106123

# (2) 컬럼(필드) 목록
print(contracts.column_names)
# → ['text']

# (3) 실제 내용 보기
print(contracts[0]['text'][:300])  # 첫 문서 앞 300자


<class 'datasets.arrow_dataset.Dataset'> 650833
['text']
QuickLinks -- Click here to rapidly navigate through this document





AMENDED AND RESTATED
EMPLOYMENT AND NONCOMPETITION AGREEMENT

    THIS AMENDED AND RESTATED EMPLOYMENT AND NONCOMPETITION AGREEMENT (the
"Agreement") is made and entered into as of October 31, 2000, by and among
Avocent Employme


In [20]:
from datasets import load_dataset, load_from_disk

# (1) 원본 로드
raw = load_dataset(
    "albertvillanova/legal_contracts",
    split="train",
    streaming=False,
    trust_remote_code=True
)

# (2) 전처리된 로컬 캐시 로드
parsed = load_from_disk("contracts_parsed")

# (3) 행 수 비교
print("원본 샘플 수:", len(raw))
print("전처리 샘플 수:", len(parsed))

# (4) 컬럼 비교
print("원본 컬럼:", raw.column_names)       # ['text']
print("전처리 컬럼:", parsed.column_names)  # ['title','date','party1','party2']

# (5) 샘플 내용 간단 비교
print("\n─── 원본[0] ───")
print(raw[0]["text"][:200])
print("\n─── 전처리[0] ───")
print(parsed[0])


원본 샘플 수: 650833
전처리 샘플 수: 650833
원본 컬럼: ['text']
전처리 컬럼: ['title', 'date', 'party1', 'party2']

─── 원본[0] ───
QuickLinks -- Click here to rapidly navigate through this document





AMENDED AND RESTATED
EMPLOYMENT AND NONCOMPETITION AGREEMENT

    THIS AMENDED AND RESTATED EMPLOYMENT AND NONCOMPETITION AGREEM

─── 전처리[0] ───
{'title': 'QuickLinks -- Click here to rapidly navigate through this document', 'date': None, 'party1': None, 'party2': None}


In [23]:
import csv

# (1) escapechar 지정
df.to_csv(
    "contracts_parsed.csv",
    index=False,
    encoding="utf-8-sig",
    escapechar="\\",            # 백슬래시로 이스케이프
    quoting=csv.QUOTE_MINIMAL   # 기본(필요할 때만 따옴표)
)


In [24]:
ds.to_json(
    "contracts_parsed.jsonl",
    orient="records",
    lines=True
)


Creating json from Arrow format: 100%|██████████| 651/651 [00:01<00:00, 620.97ba/s]


97540674

In [26]:
# 전처리 전 데이터 json화 
# 스트리밍 모드로 일부만 뽑아 테스트
stream = load_dataset(
    "albertvillanova/legal_contracts",
    split="train",
    streaming=True,
    trust_remote_code=True
)

# 첫 5개 샘플 JSON 출력
for ex in stream.take(5):
    print(ex)   # dict 형태로 {'text': ...}

    split="train",
    streaming=False,         # 전체 인덱싱·to_json() 위해 False
    trust_remote_code=True



{'text': 'QuickLinks -- Click here to rapidly navigate through this document\n\n\n\n\n\nAMENDED AND RESTATED\nEMPLOYMENT AND NONCOMPETITION AGREEMENT\n\n\xa0\xa0\xa0\xa0THIS AMENDED AND RESTATED EMPLOYMENT AND NONCOMPETITION AGREEMENT (the\n"Agreement") is made and entered into as of October\xa031, 2000, by and among\nAvocent Employment Services Co. (formerly known as Polycon Investments,\xa0Inc.), a\nTexas corporation ("Employer"), Avocent Corporation, a Delaware corporation, and\nR. Byron Driver (the "Employee").\n\n\nRECITALS\n\n\xa0\xa0\xa0\xa0WHEREAS, the Employer is a direct or indirect subsidiary of Avocent\nCorporation engaged in the business of leasing employees to Avocent Corporation\nand its affiliates, including Apex\xa0Inc. ("Apex") and Cybex Computer Products\nCorporation ("Cybex");\n\n\xa0\xa0\xa0\xa0WHEREAS, Avocent Corporation and its affiliates (collectively referred to in\nthis Agreement as "Avocent") are engaged in the business of designing,\nmanufacturing, and sell